In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Coahuayana,MX,18.7333,-103.6833,75.51,66,41,4.14,scattered clouds
1,1,Yumen,CN,40.2833,97.2000,53.42,50,17,17.20,few clouds
2,2,Albany,US,42.6001,-73.9662,61.45,47,73,9.10,broken clouds
3,3,Esmeraldas,BR,-19.7625,-44.3139,76.06,66,100,5.77,overcast clouds
4,4,Mataura,NZ,-46.1927,168.8643,54.48,76,90,12.10,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? ")) 

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10) 


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Coahuayana,MX,18.7333,-103.6833,75.51,66,41,4.14,scattered clouds
3,3,Esmeraldas,BR,-19.7625,-44.3139,76.06,66,100,5.77,overcast clouds
5,5,Dingle,PH,10.9995,122.6711,84.61,74,94,11.01,overcast clouds
7,7,Saint George,US,37.1041,-113.5841,71.64,12,0,16.11,clear sky
9,9,Panaba,MX,21.2833,-88.2667,80.11,64,0,15.39,clear sky
14,14,Pocone,BR,-16.2567,-56.6228,81.68,63,100,4.83,overcast clouds
15,15,Atuona,PF,-9.8000,-139.0333,79.07,74,24,17.07,few clouds
16,16,Hilo,US,19.7297,-155.0900,83.28,84,100,4.00,overcast clouds
18,18,Jamestown,US,42.0970,-79.2353,73.04,40,0,5.75,clear sky
19,19,Araouane,ML,18.9048,-3.5265,76.15,23,0,9.64,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count() 

303


City_ID                303
City                   303
Country                303
Lat                    303
Lng                    303
Max Temp               303
Humidity               303
Cloudiness             303
Wind Speed             303
Current Description    303
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head() 

303


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Coahuayana,MX,18.7333,-103.6833,75.51,66,41,4.14,scattered clouds
3,3,Esmeraldas,BR,-19.7625,-44.3139,76.06,66,100,5.77,overcast clouds
5,5,Dingle,PH,10.9995,122.6711,84.61,74,94,11.01,overcast clouds
7,7,Saint George,US,37.1041,-113.5841,71.64,12,0,16.11,clear sky
9,9,Panaba,MX,21.2833,-88.2667,80.11,64,0,15.39,clear sky


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Coahuayana,MX,75.51,scattered clouds,18.7333,-103.6833,
3,Esmeraldas,BR,76.06,overcast clouds,-19.7625,-44.3139,
5,Dingle,PH,84.61,overcast clouds,10.9995,122.6711,
7,Saint George,US,71.64,clear sky,37.1041,-113.5841,
9,Panaba,MX,80.11,clear sky,21.2833,-88.2667,
14,Pocone,BR,81.68,overcast clouds,-16.2567,-56.6228,
15,Atuona,PF,79.07,few clouds,-9.8000,-139.0333,
16,Hilo,US,83.28,overcast clouds,19.7297,-155.0900,
18,Jamestown,US,73.04,clear sky,42.0970,-79.2353,
19,Araouane,ML,76.15,clear sky,18.9048,-3.5265,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key 
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10) 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Coahuayana,MX,75.51,scattered clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
3,Esmeraldas,BR,76.06,overcast clouds,-19.7625,-44.3139,Bom Será - Marcinho e Fernanda
5,Dingle,PH,84.61,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
7,Saint George,US,71.64,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
9,Panaba,MX,80.11,clear sky,21.2833,-88.2667,Hotel California
14,Pocone,BR,81.68,overcast clouds,-16.2567,-56.6228,Hotel e Churrascaria Pantaneira
15,Atuona,PF,79.07,few clouds,-9.8000,-139.0333,Villa Enata
16,Hilo,US,83.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
18,Jamestown,US,73.04,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
19,Araouane,ML,76.15,clear sky,18.9048,-3.5265,


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv" 
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 

# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig 


Figure(layout=FigureLayout(height='420px'))